## Kaggle Setup

In [ ]:
!nvidia-smi
!pip install --upgrade kaggle

Fri Nov 13 09:24:36 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.32.00    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   42C    P0    29W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
mkdir .kaggle

mkdir: cannot create directory ‘.kaggle’: File exists


In [ ]:
import json
token = {"username":"kexinzhangcmu","key":"91f327ebd62f2d3cfeb09f0344748606"}

with open('/content/.kaggle/kaggle.json', 'w') as file:
      json.dump(token, file)

In [ ]:
!chmod 600 /content/.kaggle/kaggle.json
!mkdir ~/.kaggle
!cp /content/.kaggle/kaggle.json ~/.kaggle/kaggle.json
!kaggle config set -n path -v /content/
!kaggle competitions download -c 11-785-fall-20-homework-3
%cd competitions/11-785-fall-20-homework-3
!unzip \*.zip
!rm -rf *.zip

mkdir: cannot create directory ‘/root/.kaggle’: File exists
- path is now set to: /content/
phoneme_list.py: Skipping, found more recently modified local copy (use --force to force download)
 74% 73.0M/98.1M [00:00<00:00, 92.3MB/s]
100% 98.1M/98.1M [00:00<00:00, 119MB/s] 
  0% 0.00/3.21M [00:00<?, ?B/s]
100% 3.21M/3.21M [00:00<00:00, 214MB/s]
 98% 97.0M/98.6M [00:00<00:00, 68.0MB/s]
100% 98.6M/98.6M [00:00<00:00, 127MB/s] 
 96% 1.51G/1.57G [00:12<00:00, 133MB/s]User cancelled operation
/content/competitions/11-785-fall-20-homework-3
Archive:  train.npy.zip
  End-of-central-directory signature not found.  Either this file is not
  a zipfile, or it constitutes one disk of a multi-part archive.  In the
  latter case the central directory and zipfile comment will be found on
  the last disk(s) of this archive.
Archive:  dev.npy.zip
replace dev.npy? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: dev.npy                 y


Archive:  train_labels.npy.zip
replace train_labels.npy? [y]es

## Install CTC Decode

In [ ]:
%cd /content
!git clone --recursive https://github.com/parlance/ctcdecode.git
!pip install wget
%cd ctcdecode
!pip install .
%cd ..

/content
fatal: destination path 'ctcdecode' already exists and is not an empty directory.
/content/ctcdecode
Processing /content/ctcdecode


## Mount GDrive

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
ls

## Imports

In [ ]:
!cp /content/competitions/11-785-fall-20-homework-3/phoneme_list.py /content/

In [ ]:
import sys
sys.path.append("/content/competitions/11-785-fall-20-homework-3/")

import torch
from torch import nn
from torch.nn.utils.rnn import *
from phoneme_list import N_PHONEMES, PHONEME_LIST, PHONEME_MAP
from torch.optim.lr_scheduler import StepLR, MultiplicativeLR, CosineAnnealingLR

from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import time

torch.__version__

from tqdm import tqdm

In [ ]:
cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

## Load Data

In [ ]:
import numpy as np
import torch

DATA_PATH = "/content/competitions/11-785-fall-20-homework-3/"

def load_data(data_path=DATA_PATH):
    """load data"""
    train = np.load(data_path + "train.npy",allow_pickle=True)
    train_labels = np.load(data_path + "train_labels.npy",allow_pickle=True)
    dev = np.load(data_path + 'dev.npy', allow_pickle=True)
    dev_labels = np.load(data_path + 'dev_labels.npy', allow_pickle=True)
    
    return train, train_labels, dev, dev_labels 


train, train_labels, dev, dev_labels = load_data(DATA_PATH)
print("training samples:", train.shape[0], "dev samples:", dev.shape[0])

training samples: 22002 dev samples: 2332


In [ ]:
def load_test_data(data_path=DATA_PATH):
    """load data"""
    test = np.load(data_path + "test.npy",allow_pickle=True)
    
    return test
test = load_test_data()
print("test samples:", test.shape)

test samples: (2251,)


In [ ]:
train_batch_size = 64
dev_batch_size = 64

In [ ]:
class hw3Dataset(Dataset):
    def __init__(self, X, Y):
        assert(len(X) == len(Y))
        self.X_lens = torch.LongTensor([len(x) for x in X])
        train_tensors = [torch.FloatTensor(x).float() for x in X]
        self.X = pad_sequence(train_tensors, batch_first=True)

        self.Y_lens = torch.LongTensor([len(y) for y in Y])
        labels_tensors = [torch.LongTensor(y + 1) for y in Y] # add 1
        self.Y = pad_sequence(labels_tensors, batch_first=True)

        print("# of examples:", self.X.shape, "//", self.Y.shape)
        print("max:", self.Y.max(), "min:", self.Y.min())
    
    def __len__(self):
        return len(self.Y)
    
    def __getitem__(self, index):
        X = self.X[index]
        Y = self.Y[index]
        X_lens = self.X_lens[index]
        Y_lens = self.Y_lens[index]                
        return X, X_lens, Y, Y_lens
    

num_workers = 4

train_dataset = hw3Dataset(train, train_labels)
train_loader_args = dict(shuffle = True, batch_size = train_batch_size,
                        num_workers = num_workers, pin_memory = True, drop_last = False)
train_loader = DataLoader(train_dataset, **train_loader_args)

# of examples: torch.Size([22002, 2450, 13]) // torch.Size([22002, 274])
max: tensor(41) min: tensor(0)


In [ ]:
dev_dataset = hw3Dataset(dev, dev_labels)
dev_loader_args = dict(shuffle = False, batch_size = dev_batch_size,
                        num_workers = num_workers, pin_memory = True, drop_last = False)
dev_loader = DataLoader(dev_dataset, **dev_loader_args)

# generating true labels
Y_true_list = []
Y_lens_true_list = []

for idx, (X, X_lens, Y, Y_lens) in tqdm(enumerate(dev_loader)):
    X, X_lens, Y, Y_lens = X.to(device), X_lens.to(device), Y.to(device), Y_lens.to(device)
    Y_true_list.extend(Y.cpu().numpy())
    Y_lens_true_list.extend(Y_lens.cpu().numpy())

true_string_list = []
for y_true, lens_true in tqdm(zip(Y_true_list, Y_lens_true_list)):
    true_seq = y_true[:lens_true]
    true_string = ''.join(PHONEME_MAP[i] for i in true_seq)
    true_string_list.append(true_string)

# of examples: torch.Size([2332, 3246, 13]) // torch.Size([2332, 364])
max: tensor(41) min: tensor(0)


37it [00:05,  7.35it/s]
2332it [00:00, 64426.17it/s]


In [ ]:
class Model(nn.Module):
    def __init__(self, hidden_size, embed_size = 13, out_vocab = 42):
        super(Model, self).__init__()
        self.lstm = nn.LSTM(input_size=embed_size, hidden_size=hidden_size, 
                            bidirectional=True, num_layers=5)
        self.output = nn.Sequential(
            nn.Linear(hidden_size * 2, hidden_size * 2),
            # nn.ReLU()
            nn.LeakyReLU(),
            nn.Dropout(0.1),
            # nn.BatchNorm1d(hidden_size),
            nn.Linear(hidden_size * 2, out_vocab))

    
    def forward(self, X, lengths):
        packed_X = pack_padded_sequence(X, lengths, enforce_sorted=False, batch_first = True)
        packed_out = self.lstm(packed_X)[0]
        out, out_lens = pad_packed_sequence(packed_out)
        # Log softmax after output layer is required since`nn.CTCLoss` expects log probabilities.
        out = self.output(out).log_softmax(2)
        return out, out_lens


In [ ]:
# load model
model = Model(512)
model.train()
model.to(device)

criterion = nn.CTCLoss()

In [ ]:
# min_val_loss = float(model_file[-4:])
save_path = "/content/gdrive/My Drive/11685deeplearning/hw3p2/"
model_name = "wider_2_baseline"

In [ ]:
def validate(model, data_loader, criterion, device, min_val_loss):
    model.eval()
    val_loss = []
    with torch.no_grad():
        for idx, (X, X_lens, Y, Y_lens) in enumerate(data_loader):

            X, X_lens, Y, Y_lens = X.to(device), X_lens.to(device), Y.to(device), Y_lens.to(device)

            optimizer.zero_grad()
            out, out_lens = model(X, X_lens.cpu())
            loss = criterion(out, Y, out_lens, Y_lens)
            val_loss.append(loss.item())

            del X, X_lens, Y, Y_lens

    model.train()
    mean_val_loss = np.mean(val_loss)

    if mean_val_loss < min_val_loss:
        print("save model!")
        torch.save({
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict()}, 
        save_path + model_name + "_" + str(epoch) + "_" + "{:.2f}".format(mean_val_loss))

        min_val_loss = val_loss
    return mean_val_loss

#### Multiplicative LR

In [ ]:
weightDecay = 1e-5
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3, weight_decay = weightDecay)
# MultiplicativeLR
lmbda = lambda epoch: 0.85
scheduler = MultiplicativeLR(optimizer, lr_lambda=lmbda)

In [ ]:
start = time.time()
numEpochs = 10 # TODO: change num of epochs

for epoch in range(numEpochs):
    for idx, (X, X_lens, Y, Y_lens) in enumerate(train_loader):
        model.train()
        X, X_lens, Y, Y_lens = X.to(device), X_lens.to(device), Y.to(device), Y_lens.to(device)

        optimizer.zero_grad()
        out, out_lens = model(X, X_lens.cpu())
        loss = criterion(out, Y, out_lens, Y_lens)

        del X, X_lens, Y, Y_lens
        del out, out_lens
        torch.cuda.empty_cache()

        loss.backward()
        optimizer.step()

        if idx % 10 == 9:
            print('Epoch: {}/Batch: {}\t Train Loss: {:.4f}\ttime: {}sec'.format(epoch + 1, idx + 1,
                                                                            loss.item(),
                                                                            int(time.time() - start)))
            start = time.time()
            # print("Learning rate: ", scheduler.get_last_lr()[0])

        torch.cuda.empty_cache()
        # scheduler.step() # TODO: uncomment if scheduler = cosineAnnealing
        
    val_loss = validate(model, dev_loader, criterion, device, min_val_loss)
    print('Epoch: {}\tTrain Loss: {:.4f}\tVal Loss: {:.4f}'.format(epoch + 1,loss.item(), val_loss))

    scheduler.step()
    print("Learning rate: ", scheduler.get_last_lr()[0])

In [ ]:
weightDecay = 1e-5
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4, weight_decay = weightDecay)
# MultiplicativeLR
lmbda = lambda epoch: 0.85
scheduler = MultiplicativeLR(optimizer, lr_lambda=lmbda)

In [ ]:
start = time.time()
numEpochs = 10 # TODO: change num of epochs

for epoch in range(numEpochs):
    for idx, (X, X_lens, Y, Y_lens) in enumerate(train_loader):
        model.train()
        X, X_lens, Y, Y_lens = X.to(device), X_lens.to(device), Y.to(device), Y_lens.to(device)

        optimizer.zero_grad()
        out, out_lens = model(X, X_lens.cpu())
        loss = criterion(out, Y, out_lens, Y_lens)

        del X, X_lens, Y, Y_lens
        del out, out_lens
        torch.cuda.empty_cache()

        loss.backward()
        optimizer.step()

        if idx % 10 == 9:
            print('Epoch: {}/Batch: {}\t Train Loss: {:.4f}\ttime: {}sec'.format(epoch + 1, idx + 1,
                                                                            loss.item(),
                                                                            int(time.time() - start)))
            start = time.time()
            # print("Learning rate: ", scheduler.get_last_lr()[0])

        torch.cuda.empty_cache()
        # scheduler.step() # TODO: uncomment if scheduler = cosineAnnealing
        
    val_loss = validate(model, dev_loader, criterion, device, min_val_loss)
    print('Epoch: {}\tTrain Loss: {:.4f}\tVal Loss: {:.4f}'.format(epoch + 1,loss.item(), val_loss))
    # print_abit(out, out_lens)

    scheduler.step()
    print("Learning rate: ", scheduler.get_last_lr()[0])

Epoch: 1/Batch: 10	 Train Loss: 3.4257	time: 34sec


KeyboardInterrupt: ignored

#### CosineAnnealing LR

In [ ]:
weightDecay = 5e-6
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5, weight_decay = weightDecay)
# CosineAnnealingLR
scheduler = CosineAnnealingLR(optimizer, len(train_loader), eta_min = 1e-7)

In [ ]:
start = time.time()
min_val_loss = 0.6
numEpochs = 5 # TODO: change num of epochs

for epoch in range(numEpochs):
    for idx, (X, X_lens, Y, Y_lens) in enumerate(train_loader):
        model.train()
        X, X_lens, Y, Y_lens = X.to(device), X_lens.to(device), Y.to(device), Y_lens.to(device)

        optimizer.zero_grad()
        out, out_lens = model(X, X_lens.cpu())
        loss = criterion(out, Y, out_lens, Y_lens)

        del X, X_lens, Y, Y_lens
        del out, out_lens
        torch.cuda.empty_cache()

        loss.backward()
        optimizer.step()

        if idx % 10 == 9:
            print('Epoch: {}/Batch: {}\t Train Loss: {:.4f}\ttime: {}sec'.format(epoch + 1, idx + 1,
                                                                            loss.item(),
                                                                            int(time.time() - start)))
            start = time.time()

        torch.cuda.empty_cache()
        scheduler.step() # TODO: uncomment if scheduler = cosineAnnealing
        
    val_loss = validate(model, dev_loader, criterion, device, min_val_loss)
    print('Epoch: {}\tTrain Loss: {:.4f}\tVal Loss: {:.4f}'.format(epoch + 1,loss.item(), val_loss))

    # scheduler.step()
    print("Learning rate: ", scheduler.get_last_lr()[0])

    print('Reset scheduler')
    scheduler = CosineAnnealingLR(optimizer, len(train_loader))

## Evaluate

In [ ]:
!pip install python-Levenshtein

     |████████████████████████████████| 51kB 2.8MB/s 
  Created wheel for python-Levenshtein: filename=python_Levenshtein-0.12.0-cp36-cp36m-linux_x86_64.whl size=144789 sha256=60c84b39fe737183ff8aa1c6ada750ea323d2d8ab6cb10a802f98ef6b3883cf2
  Stored in directory: /root/.cache/pip/wheels/de/c2/93/660fd5f7559049268ad2dc6d81c4e39e9e36518766eaf7e342
Successfully built python-Levenshtein


In [ ]:
"""
decode
"""
from ctcdecode import CTCBeamDecoder
import os
import pandas as pd

from Levenshtein import distance as levenshtein_distance
from tqdm import tqdm

dev_beam_width = 20
submit_beam_width = 100

In [ ]:
# load model
model = Model(512)
model_file = "/content/gdrive/My Drive/11685deeplearning/hw3p2/wider_2_baseline_4_0.49"
temp = torch.load(model_file)
model.load_state_dict(temp['model_state_dict'])
model.to(device)


Model(
  (lstm): LSTM(13, 512, num_layers=5, bidirectional=True)
  (output): Sequential(
    (0): Linear(in_features=1024, out_features=1024, bias=True)
    (1): LeakyReLU(negative_slope=0.01)
    (2): Dropout(p=0.1, inplace=False)
    (3): Linear(in_features=1024, out_features=42, bias=True)
  )
)

In [ ]:
"""
dev distance
"""

def validate_distance(model, dev_loader, criterion, device, dev_beam_width = dev_beam_width):
    decoder = CTCBeamDecoder(PHONEME_LIST, beam_width=dev_beam_width, 
                            num_processes=os.cpu_count(), log_probs_input=True)

    model.eval()

    Y_pred_list = []
    Y_lens_pred_list = []
    # loop the dev data loader and get beam search result
    with torch.no_grad():
        for idx, (X, X_lens, Y, Y_lens) in tqdm(enumerate(dev_loader)):
            X, X_lens, Y, Y_lens = X.to(device), X_lens.to(device), Y.to(device), Y_lens.to(device)
            out, out_lens = model(X, X_lens.cpu())
            loss = criterion(out, Y, out_lens, Y_lens)

            test_Y, _, _, test_Y_lens = decoder.decode(out.transpose(0, 1), out_lens)
            Y_pred_list.extend(test_Y[:,0,:].cpu().numpy())
            Y_lens_pred_list.extend(test_Y_lens[:,0].cpu().numpy())

    assert(len(Y_pred_list) == len(Y_lens_pred_list) == len(Y_true_list) == len(Y_lens_true_list))

    # calculate distance between true label
    pred_string_list = [] # string list for submission
    distance = [] # distance list for evaluation
    for idx, (y_pred, lens_pred) in tqdm(enumerate(zip(Y_pred_list,Y_lens_pred_list))):
        best_seq = y_pred[:lens_pred]
        pred_string = ''.join(PHONEME_MAP[i] for i in best_seq)
        pred_string_list.append(pred_string)

        distance.append(levenshtein_distance(pred_string, true_string_list[idx]))

    print("average distance: ", np.mean(distance))

validate_distance(model, dev_loader, criterion, device)



0it [00:00, ?it/s]
1it [00:02,  2.84s/it]
2it [00:04,  2.59s/it]
3it [00:08,  2.79s/it]
4it [00:10,  2.74s/it]
5it [00:13,  2.79s/it]
6it [00:16,  2.85s/it]
7it [00:18,  2.60s/it]
8it [00:21,  2.80s/it]
9it [00:24,  2.87s/it]
10it [00:26,  2.51s/it]
11it [00:29,  2.67s/it]
12it [00:31,  2.45s/it]
13it [00:34,  2.52s/it]
14it [00:37,  2.89s/it]
15it [00:41,  2.94s/it]
16it [00:43,  2.64s/it]
17it [00:46,  2.81s/it]
18it [00:49,  3.00s/it]
19it [00:52,  2.81s/it]
20it [00:54,  2.61s/it]
21it [00:58,  3.14s/it]
22it [01:01,  2.97s/it]
23it [01:03,  2.87s/it]
24it [01:05,  2.64s/it]
25it [01:07,  2.49s/it]
26it [01:10,  2.46s/it]
27it [01:12,  2.38s/it]
28it [01:14,  2.24s/it]
29it [01:17,  2.42s/it]
30it [01:19,  2.42s/it]
31it [01:22,  2.42s/it]
32it [01:24,  2.36s/it]
33it [01:27,  2.53s/it]
34it [01:29,  2.39s/it]
35it [01:31,  2.33s/it]
36it [01:34,  2.56s/it]
37it [01:36,  2.60s/it]

2332it [00:00, 41254.17it/s]

average distance:  9.440394511149227


## Make submissions

In [ ]:
# test data loader
test_labels = np.zeros((test.shape[0], 1))
test_dataset = hw3Dataset(test, test_labels)
test_loader_args = dict(shuffle = False, batch_size = 128,
                        num_workers = num_workers, pin_memory = True)
test_loader = DataLoader(test_dataset, **test_loader_args)

# of examples: torch.Size([2251, 3493, 13]) // torch.Size([2251, 1])
max: tensor(1) min: tensor(1)


In [ ]:
"""
submit
"""


def submit_csv(model, test_loader, criterion, device, submit_beam_width = submit_beam_width):
    model.eval()
    Y_pred_list = []
    Y_lens_pred_list = []


    from tqdm import tqdm
    decoder = CTCBeamDecoder(PHONEME_LIST, beam_width=submit_beam_width, 
                            num_processes=os.cpu_count(), log_probs_input=True)
    # loop the dev data loader and get beam search result
    with torch.no_grad():
        for idx, (X, X_lens, _, _) in tqdm(enumerate(test_loader)):
            X, X_lens = X.to(device), X_lens.to(device)
            out, out_lens = model(X, X_lens.cpu())

            test_Y, _, _, test_Y_lens = decoder.decode(out.transpose(0, 1), out_lens)
            Y_pred_list.extend(test_Y[:,0,:].cpu().numpy())
            Y_lens_pred_list.extend(test_Y_lens[:,0].cpu().numpy())

    assert(len(Y_pred_list) == len(Y_lens_pred_list))

    # calculate distance between true label
    pred_string_list = [] # string list for submission
    distance = [] # distance list for evaluation
    for idx, (y_pred, lens_pred) in tqdm(enumerate(zip(Y_pred_list,Y_lens_pred_list))):
        best_seq = y_pred[:lens_pred]
        pred_string = ''.join(PHONEME_MAP[i] for i in best_seq)
        pred_string_list.append(pred_string)

    df = pd.DataFrame(enumerate(pred_string_list), columns = ['id', 'label'])
    df.to_csv("submission_hw3p2.csv", index=False)

submit_csv(model, test_loader, criterion, device)


0it [00:00, ?it/s]
1it [00:27, 27.57s/it]
2it [00:56, 28.10s/it]
3it [01:14, 24.89s/it]

KeyboardInterrupt: ignored